<a href="https://colab.research.google.com/github/chen2369/data-course-sample/blob/main/%E4%BD%9C%E6%A5%AD1_%E5%AF%A6%E4%BD%9C%E3%80%8Crule_based%E3%80%8D%E7%9A%84%E6%8E%A8%E8%96%A6%E7%B3%BB%E7%B5%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Code

## 基礎建設

In [ ]:
import pandas as pd
import gzip, json
import warnings
warnings.filterwarnings('ignore')

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2021-12-24 17:19:36--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv’

All_Beauty.csv      100%[===================>]  14.78M  65.1MB/s    in 0.2s    

2021-12-24 17:19:36 (65.1 MB/s) - ‘All_Beauty.csv’ saved [15499476/15499476]

--2021-12-24 17:19:36--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz’

meta_All_Beauty.jso 100%[===================>]   9.85M  53.3MB/s    in 0.2s    

202

In [ ]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [ ]:
print(metadata.shape)
metadata.sample(5)

(32892, 19)


,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
25355,[],,[],,Malloom 1pc Nail Stamping Plates Fashion Polis...,[],,,[],"891,292 in Beauty & Personal Care (",[],{'ASIN: ': 'B017UAO4WC'},All Beauty,,,,B017UAO4WC,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
1004,[],,"[Gillette M3Power Nitro, a Mach3 innovation, i...",,Gillette M3 Power Nitro Razor,[],,Gillette,[],[],"[B00AFCPAAS, B00APFDGJM, B07DJ212QJ, B077BSS9L...",{' Product Dimensions: ': '4 x 7.9 x ...,All Beauty,,,,B0009W7MG4,[],[]
18323,[],,[],,Coscelia Acrylic Powder Liquid Kits Nail Art S...,[],,,[],"2,677,274 in Beauty & Personal Care (",[],{'ASIN: ': 'B00RG8DXYE'},All Beauty,,,,B00RG8DXYE,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
2380,[],,[BEL PREMIUM bastoncillos 300 pz],,Bel Premium Wattest&auml;bchen 300 Stk. Box,[],,BEL,[],"2,043,346 in Beauty & Personal Care (",[],"{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$17.16,B0010VI1IQ,[],[]
32617,[],,[Introducing new kids flossers from Plackers. ...,,Plackers Kids Flossers with Fluoride Fruit Smo...,"[B01IABK7RQ, B01I407CIU, B01LYQPATI, B01FRI6N5...",,,[],"276,288 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.6 ounces (', 'ASIN: ':...",All Beauty,,,$4.48,B01H440OLE,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [ ]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [ ]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

In [ ]:
import numpy as np

def emptylist_na(cell):
    try:
        if len(cell) == 0:
            return np.nan
        else:
            return cell
    except:
        return cell
metadata_na = metadata.applymap(emptylist_na)

In [ ]:
metadata_na.isna().sum().sort_values(ascending=False).to_frame(name='Number of NA').T

,category,tech2,fit,tech1,date,feature,similar_item,also_buy,also_view,price,description,imageURLHighRes,imageURL,brand,rank,details,title,main_cat,asin
Number of NA,32892,32892,32892,32882,32873,32623,31588,26295,24760,21433,17773,16351,16351,15673,377,132,1,0,0


In [ ]:
print('# main_cat:', metadata['main_cat'].unique())
print('# similar_item:', metadata['similar_item'].unique())
print('# details:', metadata['details'].sample(5))

# main_cat: ['All Beauty']
# similar_item: [''
 ' class="a-bordered a-horizontal-stripes  a-spacing-extra-large a-size-base comparison_table">\n\n\n\n            \n            \n            \n            \n            \n            <tr class="comparison_table_image_row">\n                <td class="comparison_table_first_col"></td>\n\n\n                <th class="comparison_image_title_cell" role="columnheader">\n                    <div class="a-row a-spacing-top-micro">\n                        <center>\n                             <img alt="No7 Lift &amp; Luminate Triple Action Serum 50ml by Boots" src="https://images-na.ssl-images-amazon.com/images/I/31gXVsxM7FL._SL500_AC_SS350_.jpg" id="comparison_image">\n                        </center>\n                    </div>\n                    <div class="a-row a-spacing-top-small">\n                        <div id="comparison_title" class="a-section a-spacing-none">\n                            <span aria-hidden="true" class="a-size-b

In [ ]:
metadata_clean = metadata[['asin', 'rank', 'brand', 'price', 'also_view', 'also_buy']]
metadata_clean.head()

,asin,rank,brand,price,also_view,also_buy
0,6546546450,"2,938,573 in Beauty & Personal Care (",idea village,,[],[]
1,7178680776,"872,854 in Beauty & Personal Care (",,$44.99,[],"[B01E7LCSL6, B008X5RVME]"
2,7250468162,"956,696 in Beauty & Personal Care (",No7,$28.76,"[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]",[]
3,7367905066,"1,870,258 in Beauty & Personal Care (",,,[],[B0041PBXX8]
4,7414204790,"67,701 in Beauty & Personal Care (",Pirmal Healthcare,$12.15,"[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...",[]


In [ ]:
metadata_clean['price'] = metadata_clean['price'].str.replace('$', '')
metadata_clean['price'] = pd.to_numeric(metadata_clean['price'], errors = 'coerce')

In [ ]:
metadata_clean['cate'] = metadata_clean['rank'].str.split(' in ').str[1]
metadata_clean['cate'].unique()

array(['Beauty & Personal Care (', 'Beauty &amp; Personal Care (', nan,
       'Grocery & Gourmet Food (', 'Grocery &amp; Gourmet Food (',
       'Health &amp; Household (', 'Sports & Outdoors (',
       'Health & Household (', 'Toys & Games (', 'Baby (',
       'Clothing, Shoes & Jewelry (', 'Home & Kitchen (', 'Automotive (',
       'Tools & Home Improvement ('], dtype=object)

In [ ]:
metadata_clean['cate'] = metadata_clean['cate'].str.replace('amp;', '')
metadata_clean['cate'] = metadata_clean['cate'].str.replace(' \(', '')
metadata_clean['cate'].unique()

array(['Beauty & Personal Care', nan, 'Grocery & Gourmet Food',
       'Health & Household', 'Sports & Outdoors', 'Toys & Games', 'Baby',
       'Clothing, Shoes & Jewelry', 'Home & Kitchen', 'Automotive',
       'Tools & Home Improvement'], dtype=object)

In [ ]:
metadata_clean['rank'] = metadata_clean['rank'].str.split(expand = True)[0]
metadata_clean['rank'] = metadata_clean['rank'].str.replace(',', '')
metadata_clean['rank'] = pd.to_numeric(metadata_clean['rank'])

In [ ]:
metadata_clean

,asin,rank,brand,price,also_view,also_buy,cate
0,6546546450,2938573.0,idea village,NaN,[],[],Beauty & Personal Care
1,7178680776,872854.0,,44.99,[],"[B01E7LCSL6, B008X5RVME]",Beauty & Personal Care
2,7250468162,956696.0,No7,28.76,"[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]",[],Beauty & Personal Care
3,7367905066,1870258.0,,NaN,[],[B0041PBXX8],Beauty & Personal Care
4,7414204790,67701.0,Pirmal Healthcare,12.15,"[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...",[],Beauty & Personal Care
...,...,...,...,...,...,...,...
32887,B01HIWLLUK,2145325.0,,9.95,[],[],Beauty & Personal Care
32888,B01HJ1K3YK,1639713.0,Salon Perfect,NaN,[],[],Beauty & Personal Care
32889,B01HJ84SGM,207410.0,,55.63,"[B01KON9B4S, B000JN4CR0, B071ZHMRHS, B01HJ84TN...","[B01KON9B4S, B079X3YFXS, B00M79OYS6, B000JN4CR...",Beauty & Personal Care
32890,B01HJASD20,965673.0,GBSTORE,12.99,[],[],Beauty & Personal Care


## 資料切分

In [ ]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

In [ ]:
ratings_trainings['asin'].value_counts().describe()

count    32582.000000
mean        11.379044
std         97.311099
min          1.000000
25%          1.000000
50%          2.000000
75%          5.000000
max       8672.000000
Name: asin, dtype: float64

In [ ]:
ratehot_table = ratings_trainings.groupby(by = ['asin']).aggregate({'overall':['mean', 'size']})['overall']

In [ ]:
ratings_trainings = pd.merge(ratings_trainings, ratehot_table, on='asin', how='left')

In [ ]:
ratings_trainings_drop = ratings_trainings.drop_duplicates()

In [ ]:
print(ratings_trainings_drop[['asin', 'reviewerID']].value_counts())
ratings_trainings_drop[['asin', 'reviewerID']].value_counts()[ratings_trainings_drop[['asin', 'reviewerID']].value_counts()>1]

asin        reviewerID    
B00W259T7G  A1EGCED01USBA9    7
            A1210QJT54O8T0    5
B00BMVV3MK  A3CD2XV17M8FWK    4
B00W259T7G  A2UEIN7SIPZFRP    4
            AAVHTSNM003DH     4
                             ..
B00SQ1NQAG  A2K0AI16P39WPX    1
            A2YC2UBU204LY1    1
            A2YSAX8Z45XY1I    1
            A31QY2E8JUQQT9    1
0061073717  A2XMFX1BR0IJFJ    1
Length: 361012, dtype: int64


asin        reviewerID    
B00W259T7G  A1EGCED01USBA9    7
            A1210QJT54O8T0    5
B00BMVV3MK  A3CD2XV17M8FWK    4
B00W259T7G  A2UEIN7SIPZFRP    4
            AAVHTSNM003DH     4
                             ..
            A2XAKGRUHIE2ZA    2
B002K5SWA8  A3FBOBYK2BQYIB    2
B00W259T7G  A2EQ29FCIIXPUG    2
B00FE90M0K  A3UYMUP46P1K5X    2
B00W259T7G  A2D2M8D7Y3WXSS    2
Length: 623, dtype: int64

In [ ]:
ratings_trainings[ratings_trainings['reviewerID'] == 'A1210QJT54O8T0'].sort_values(['DATE'], ascending=False)

,asin,reviewerID,overall,unixReviewTime,DATE,mean,size
339675,B015Y0NAD4,A1210QJT54O8T0,5.0,1489017600,2017-03-09,5.000000,7
193879,B00W259T7G,A1210QJT54O8T0,5.0,1414195200,2014-10-25,4.516729,2959
193965,B00W259T7G,A1210QJT54O8T0,5.0,1406764800,2014-07-31,4.516729,2959
194101,B00W259T7G,A1210QJT54O8T0,5.0,1391212800,2014-02-01,4.516729,2959
194102,B00W259T7G,A1210QJT54O8T0,5.0,1391212800,2014-02-01,4.516729,2959
194103,B00W259T7G,A1210QJT54O8T0,5.0,1391212800,2014-02-01,4.516729,2959
194154,B00W259T7G,A1210QJT54O8T0,5.0,1383523200,2013-11-04,4.516729,2959
194172,B00W259T7G,A1210QJT54O8T0,5.0,1381276800,2013-10-09,4.516729,2959
270758,B003C5FLTK,A1210QJT54O8T0,5.0,1318550400,2011-10-14,5.000000,2


In [ ]:
ratings_trainings['overall'].describe()

count    370752.000000
mean          4.112326
std           1.361899
min           1.000000
25%           4.000000
50%           5.000000
75%           5.000000
max           5.000000
Name: overall, dtype: float64

In [ ]:
print('brand:', len(metadata_clean['brand'].unique())-1) # nan
print('cate:', len(metadata_clean['cate'].unique())-1) # nan

brand: 7862
cate: 10


## 產生推薦

## 1

In [ ]:
def recommender(training_data, users=[], k=10, rule='rule-based', days=9999):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    global metadata_clean

    training_data = training_data[(training_data['DATE'] + datetime.timedelta(days = days)) > '2018-09-01']
    ratehot_table = training_data.groupby(by = ['asin']).aggregate({'overall':['mean', 'size']})['overall']
    metadata_clean1 = pd.merge(metadata_clean, ratehot_table, on='asin', how='left')
    metadata_clean1 = metadata_clean1.sort_values(['size'], ascending=False)

    recommendations = {}
    reco = []

    for user in users:
        if rule == 'rule-based':

            buy = []
            view = []
            other = []

            user_review = training_data[training_data['reviewerID'] == user].sort_values(['size', 'DATE'], ascending=[False, False]).drop_duplicates(subset='asin', keep='first')

            for index, row in user_review.iterrows():
                if row['overall'] == 5:
                    buy.extend(row['asin'])
        
                meta = metadata_clean1[metadata_clean1['asin'] == row['asin']].iloc[0]
                if len(meta['also_buy']) > 0:
                    buy.extend(meta['also_buy'])
                if len(meta['also_view']) > 0:
                    view.extend(meta['also_view'])

                cate = meta['cate']
                brand = meta['brand']
                if len(brand) > 0:
                    df = metadata_clean1[(metadata_clean1['brand'] == brand) & (metadata_clean1['cate'] == cate)]
                    other.extend(df['asin'].tolist())
                    df = metadata_clean1[:int(metadata_clean1.shape[0]*0.01*k)][metadata_clean1['brand'] == brand]
                    other.extend(df['asin'].tolist())
                df = metadata_clean1[:int(metadata_clean1.shape[0]*0.00005*k)][metadata_clean1['cate'] == cate]
                other.extend(df['asin'].tolist())

            df_buy = metadata_clean1[metadata_clean1.asin.isin(buy)]
            reco.extend(df_buy['asin'].tolist())
            df_view = metadata_clean1[metadata_clean1.asin.isin(view)]
            reco.extend(df_view['asin'].tolist())                

            df = metadata_clean1[:int(metadata_clean1.shape[0]*0.00005*k)]
            other.extend(df['asin'].tolist())
            other = list(set(other))            
            reco.extend(other)

        recommendations[user] = reco[:k]
    return recommendations

# ratings_by_user = recommender(ratings_trainings, users)
# ratings_by_user

## 2

In [ ]:
import datetime

In [ ]:
def recommender_without_user(training_data, users=[], k=10, rule='rule-based', days=9999):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    global metadata_clean

    training_data = training_data[(training_data['DATE'] + datetime.timedelta(days = days)) > '2018-09-01']
    ratehot_table = training_data.groupby(by = ['asin']).aggregate({'overall':['mean', 'size']})['overall']
    metadata_clean2 = pd.merge(metadata_clean, ratehot_table, on='asin', how='left')
    metadata_clean2 = metadata_clean2.sort_values(['size'], ascending=False)


    recommendations = {}
    reco = []
    if rule == 'random':
        reco = metadata_clean2['asin'].sample(k).tolist()
    elif rule == 'hottest':
        reco = training_data.sort_values(['size'], ascending=False)['asin'].unique()[:k]
    elif rule == 'top10':
        top10 = metadata_clean2[metadata_clean2['rank'] <= metadata_clean2.groupby('cate')['rank'].transform('quantile', .1)]
        for index, row in top10.iterrows():
            reco.extend([row['asin']])
            # reco.extend(row['also_buy'])
            if len(reco) >= k:
                break
    for user in users:
        recommendations[user] = reco[:k]
    return recommendations

In [ ]:
recommender_without_user(ratings_trainings, users, k=50, rule='top10', days=100)

## 結果評估

In [ ]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

## 結果比較

### 依照用戶過去購買資料

In [ ]:
klst = range(10,51,10)
dayslst = [5, 10, 30, 60, 90, 180, 360, 720]
scorelst = []
for k in klst:
    for days in dayslst:    
        ratings_by_user = recommender(ratings_trainings, users, k=k, rule='rule-based', days=days)
        score = evaluate(ratings_testings_by_user, ratings_by_user)
        scorelst.append(score)
        print("k = %d, days = %d, score = %.5f" %(k, days, score))

pd.DataFrame(np.reshape(scorelst, (5, 8)).T, index=pd.Index(dayslst, name='Days'), columns=pd.Index(klst, name='k'))

k = 10, days = 5, score = 0.15932
k = 10, days = 10, score = 0.15763
k = 10, days = 30, score = 0.14237
k = 10, days = 60, score = 0.14068
k = 10, days = 90, score = 0.13559
k = 10, days = 180, score = 0.09831
k = 10, days = 360, score = 0.09831
k = 10, days = 720, score = 0.09831
k = 20, days = 5, score = 0.16610
k = 20, days = 10, score = 0.20678
k = 20, days = 30, score = 0.21695
k = 20, days = 60, score = 0.22373
k = 20, days = 90, score = 0.22034
k = 20, days = 180, score = 0.15085
k = 20, days = 360, score = 0.12542
k = 20, days = 720, score = 0.11186
k = 30, days = 5, score = 0.16610
k = 30, days = 10, score = 0.21695
k = 30, days = 30, score = 0.24746
k = 30, days = 60, score = 0.25085
k = 30, days = 90, score = 0.21864
k = 30, days = 180, score = 0.18644
k = 30, days = 360, score = 0.15593
k = 30, days = 720, score = 0.13898
k = 40, days = 5, score = 0.17966
k = 40, days = 10, score = 0.22203
k = 40, days = 30, score = 0.24068
k = 40, days = 60, score = 0.25932
k = 40, days = 

k,10,20,30,40,50
Days,,,,,
5,0.159322,0.166102,0.166102,0.179661,0.108475
10,0.157627,0.206780,0.216949,0.222034,0.164407
30,0.142373,0.216949,0.247458,0.240678,0.174576
60,0.140678,0.223729,0.250847,0.259322,0.183051
90,0.135593,0.220339,0.218644,0.238983,0.172881
180,0.098305,0.150847,0.186441,0.223729,0.137288
360,0.098305,0.125424,0.155932,0.189831,0.105085
720,0.098305,0.111864,0.138983,0.145763,0.054237


### 各分類中的前10％，依據購買熱度排序

In [ ]:
klst = range(10,51,10)
dayslst = [5, 10, 30, 60, 90, 180, 360, 720]
scorelst = []
for k in klst:
    for days in dayslst:    
        ratings_by_user = recommender_without_user(ratings_trainings, users, k=k, rule='top10', days=days)
        score = evaluate(ratings_testings_by_user, ratings_by_user)
        scorelst.append(score)
        # print("k = %d, days = %d, score = %.5f" %(k, days, score))

pd.DataFrame(np.reshape(scorelst, (5, 8)).T, index=pd.Index(dayslst, name='Days'), columns=pd.Index(klst, name='k'))

k,10,20,30,40,50
Days,,,,,
5,0.152542,0.166102,0.177966,0.200000,0.210169
10,0.142373,0.222034,0.237288,0.255932,0.266102
30,0.154237,0.196610,0.242373,0.291525,0.298305
60,0.150847,0.176271,0.242373,0.276271,0.308475
90,0.133898,0.189831,0.223729,0.249153,0.283051
180,0.098305,0.149153,0.176271,0.196610,0.232203
360,0.098305,0.106780,0.130508,0.162712,0.213559
720,0.083051,0.100000,0.113559,0.144068,0.157627


In [ ]:
klst = range(10,51,10)
dayslst = [5, 10, 30, 60, 90, 180, 360, 720]
scorelst = []
for k in klst:
    for days in dayslst:    
        ratings_by_user = recommender_without_user(ratings_trainings, users, k=k, rule='top10', days=days)
        score = evaluate(ratings_testings_by_user, ratings_by_user)
        scorelst.append(score)
        # print("k = %d, days = %d, score = %.5f" %(k, days, score))

pd.DataFrame(np.reshape(scorelst, (5, 8)).T, index=pd.Index(dayslst, name='Days'), columns=pd.Index(klst, name='k'))

k,10,20,30,40,50
Days,,,,,
5,0.083051,0.083051,0.083051,0.083051,0.083051
10,0.083051,0.083051,0.096610,0.096610,0.096610
30,0.083051,0.083051,0.083051,0.084746,0.084746
60,0.083051,0.083051,0.083051,0.084746,0.084746
90,0.083051,0.083051,0.083051,0.083051,0.083051
180,0.083051,0.083051,0.083051,0.096610,0.096610
360,0.083051,0.083051,0.083051,0.096610,0.096610
720,0.083051,0.083051,0.083051,0.083051,0.083051


### 依照過去購買熱度排序

In [ ]:
klst = range(10,51,10)
dayslst = [5, 10, 30, 60, 90, 180, 360, 720]
scorelst = []
for k in klst:
    for days in dayslst:    
        ratings_by_user = recommender_without_user(ratings_trainings, users, k=k, rule='hottest', days=days)
        score = evaluate(ratings_testings_by_user, ratings_by_user)
        scorelst.append(score)
        # print("k = %d, days = %d, score = %.5f" %(k, days, score))

pd.DataFrame(np.reshape(scorelst, (5, 8)).T, index=pd.Index(dayslst, name='Days'), columns=pd.Index(klst, name='k'))

k,10,20,30,40,50
Days,,,,,
5,0.113559,0.144068,0.233898,0.255932,0.262712
10,0.113559,0.147458,0.184746,0.261017,0.306780
30,0.105085,0.123729,0.155932,0.172881,0.194915
60,0.105085,0.110169,0.123729,0.147458,0.161017
90,0.083051,0.105085,0.110169,0.118644,0.140678
180,0.083051,0.083051,0.083051,0.091525,0.105085
360,0.083051,0.083051,0.083051,0.091525,0.105085
720,0.083051,0.083051,0.083051,0.091525,0.105085


### 其他觀察

- Testing Data中有多少使用者出現在Training Data中

In [ ]:
users_training = ratings_trainings['reviewerID'].tolist()
print('Appear:', len([user for user in users if user in users_training]))
print('Total:', len(users))

Appear: 38
Total: 584


#### 將K放大

In [ ]:
klst = np.linspace(10, 1000, 5, dtype=int)
dayslst = [5, 10, 30, 60, 90, 180, 360, 720]
scorelst = []
for k in klst:
    for days in dayslst:    
        ratings_by_user = recommender_without_user(ratings_trainings, users, k=k, rule='hottest', days=days)
        score = evaluate(ratings_testings_by_user, ratings_by_user)
        scorelst.append(score)
        # print("k = %d, days = %d, score = %.5f" %(k, days, score))

pd.DataFrame(np.reshape(scorelst, (5, 8)).T, index=pd.Index(dayslst, name='Days'), columns=pd.Index(klst, name='k'))

k,10,257,505,752,1000
Days,,,,,
5,0.113559,0.288136,0.288136,0.288136,0.288136
10,0.113559,0.427119,0.427119,0.427119,0.427119
30,0.105085,0.506780,0.622034,0.654237,0.654237
60,0.105085,0.418644,0.559322,0.657627,0.693220
90,0.083051,0.352542,0.484746,0.576271,0.657627
180,0.083051,0.198305,0.389831,0.466102,0.516949
360,0.083051,0.198305,0.364407,0.425424,0.484746
720,0.083051,0.196610,0.354237,0.406780,0.476271


#### 隨機推薦結果

In [ ]:
klst = np.linspace(10, metadata_clean.shape[0], 11, dtype=int)
for k in klst:
    ratings_by_user = recommender_without_user(ratings_trainings, users, k=k,rule='random')
    score = evaluate(ratings_testings_by_user, ratings_by_user)
    print("k = %d, score = %.5f" %(k, score))

k = 10, score = 0.00000
k = 3298, score = 0.06441
k = 6586, score = 0.15424
k = 9874, score = 0.28305
k = 13162, score = 0.36780
k = 16451, score = 0.51017
k = 19739, score = 0.56780
k = 23027, score = 0.79661
k = 26315, score = 0.79831
k = 29603, score = 0.89153
k = 32892, score = 1.00000
